# Simulate Sequences

In [ ]:
### IMPORTS
from simulate import run_simulation

### Run Single Simulation

In [ ]:
# Number of unique monomers
N_MONs = 4

# Molar ratios of each monomer
# ex. if N_MONs = 4: [0.5, 0.25, 0.20, 0.05]
MRs = [50, 25, 20, 5]

# What model of copolymerization to use? (1) mayolewis (terminal) or (2) penultimate
# NOTE FOR NOW ONLY MAYO-LEWIS IMPLEMENTED.
#model = "mayolewis"

# reactivity ratios of monomers, ex. for 4 monomers:
#[r12, r13, r14]
#[r21, r23, r24]
#[r31, r32, r34]
#[r41, r42, r43]
RRs = ([[1, 1, 1],
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1]])

# % conversion targetted (0-1), i.e. how much of the monomer pool do you want to use
conv = 0.5

# average degree of polymerization (chain length) you are targetting at YOUR conversion, NOT at 100%.
avgDP = 100

# number of polymer chains to simulate
N_CHAINs = 10000

# Chain transfer % (0-1)
# TODO: replace this with direct PDI control (not sure how)
# NOTE TO SELF: for now just fix CTP such that PDI ~ 1.2, as thats roughly the control our RAFT synthesis gives.
CTP = 0.05

# cutoff DP of chains considered as polymers not oligomers that get "purified" out
# set to 0 if you don't want to do any filtration
PRUNE_OLIGOMERS = 25

In [ ]:
run_simulation(N_MONs, N_CHAINs, MRs, RRs, avgDP, conv, CTP, PRUNE_OLIGOMERS)

### Run Multiple Simulations

Currently, various DPs are simulated - this can/should be changed to vary other things too, as well as instead of just simulating 4, a linspace between a max and min specifying the number of sample points.

In [ ]:
# Number of unique monomers
N_MONs = 4

# Molar ratios of each monomer
# ex. if N_MONs = 4: [0.5, 0.25, 0.20, 0.05]
MRs = [50, 25, 20, 5]

# reactivity ratios of monomers, ex. for 4 monomers:
#[r12, r13, r14]
#[r21, r23, r24]
#[r31, r32, r34]
#[r41, r42, r43]
RRs = ([[1, 1, 1],
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1]])

# % conversion targetted (0-1), i.e. how much of the monomer pool do you want to use
conv = 0.5

# average degree of polymerization (chain length) you are targetting at YOUR conversion, NOT at 100%.
avgDPs = [50, 75, 100, 125, 150, 175, 200, 225, 250]

# number of polymer chains to simulate
N_CHAINs = 10000

# Chain transfer % (0-1)
# TODO: replace this with direct PDI control (not sure how)
# NOTE TO SELF: for now just fix CTP such that PDI ~ 1.2, as thats roughly the control our RAFT synthesis gives.
CTP = 0.1

# cutoff DP of chains considered as polymers not oligomers that get "purified" out
# set to 0 if you don't want to do any filtration
PRUNE_OLIGOMERS = 25

In [ ]:
for avgDP in avgDPs:
    run_simulation(N_MONs, N_CHAINs, MRs, RRs, avgDP, conv, CTP, PRUNE_OLIGOMERS)

# Seq Analysis

In [ ]:
import pandas as pd
import numpy as np
import glob, os
import csv
import math
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import copy
import csv
from tqdm import tqdm_notebook
from itertools import tee

SMALL_SIZE = 20
MEDIUM_SIZE = 25
BIGGER_SIZE = 35

plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

### Load sequence CSV files and set monomer properties
Specify path and what monomers each label corresponds to here (by specifying Mw and HLB)

In [ ]:
### IMPORT ALL SEQUENCE CSV FILES IN FOLDER SPECIFIED BY PATH

path = "outputs"
csv_files = []
for file in os.listdir(path):
    if file.endswith(".csv"):
        csv_files.append(os.path.join(path, file))
csv_files = sorted(csv_files)
print("Number of sequence files:", len(csv_files))
print(csv_files)

### SET MOLECULAR WEIGHTS OF MONOMERS IN ORDER OF LABEL (labels are 1 indexed)
Mw = [100.121,500,198.3,246.32]
HLBs = [8.45, 11.42, 5.125, 18.5]

### Calculate general sequence statistics and average properties for all sequences by batch
Output is a dataframe with column headers: [Batch Name	NumSeqs	Avg Mn	Avg Mw	PDI	Avg DP]

In [ ]:
### CALCULATE AVERAGE DP, Mn, Mw & PDI OF ALL SEQUENCES

names = []
num_seqs = []
avg_Mns = []
avg_Mws = []
PDIs = []
avg_DPs = []
seqs = []

for m in tqdm_notebook(range(len(csv_files))):
    seq = []

    with open(csv_files[m], mode = 'r') as file:
        csvFile = csv.reader(file)
        i = 0
        for lines in csvFile:
            i = i + 1
            seq.append(lines)
    raw_seq = copy.deepcopy(seq)
    seqs.append(raw_seq)
    seqlen = np.zeros(i)
    seqweight = np.zeros(i)
    seqweight2 = np.zeros(i)

    for j in range(0,i):
        seqlen[j] = len(seq[j])
        for k in range(len(seq[j])):
            seq[j][k] = Mw[int(seq[j][k])-1]
        seqweight[j] = sum(seq[j])
        seqweight2[j] = (sum(seq[j])**2)

    AvgMw = sum(seqweight2)/sum(seqweight)
    AvgMn = sum(seqweight)/len(seqweight)
    PDI = AvgMw/AvgMn
    DP = sum(seqlen)/len(seqlen)

    names.append(csv_files[m].replace(path, '').replace('/', '').replace('.csv', ''))
    num_seqs.append(i)
    avg_Mns.append(AvgMn)
    avg_Mws.append(AvgMw)
    PDIs.append(PDI)
    avg_DPs.append(DP)
    
d = {'Batch Name': names, 'NumSeqs': num_seqs, 'Avg Mn': avg_Mns, 'Avg Mw': avg_Mws, 'PDI': PDIs, 'Avg DP': avg_DPs}
df_prop = pd.DataFrame(data=d)
df_prop

In [ ]:
#Read csv output files of 'n' batches and convert to n-element pd dataframe
dfs = []
seq_lens = []

for m in range(len(csv_files)):
    seq = []
    row_size = []
    
    with open(csv_files[m], mode = 'r') as file:
        csvFile = csv.reader(file)
        i = 0
        for lines in csvFile:
            i = i + 1
            seq.append(lines)
            row_size.append(len(lines))
        df = pd.DataFrame(seq)
        dfs.append(df)
        seq_lens.append(row_size)

## mon2mon level
Intra-chain positional variance
- visualize sequences simulated
- hydropathy plots with different window sizes
- specific segment pattern search

In [ ]:
#specific segment pattern search
b_no = 8
s_no = 837

oegma_counter = 0
seg_len = 0
seg_array = []

for i in range(len(dfs[b_no].iloc[s_no])):
    if dfs[b_no][i][s_no] == str(2):
        back_counter = 0
        forward_counter = 0
        j = 1
        k = 1
        while i-j >= 0 and (dfs[b_no][i-j][s_no] == str(1) or dfs[b_no][i-j][s_no] == str(3)):
            j += 1
            back_counter += 1
        while i+k <= len(dfs[b_no].iloc[s_no])-1 and (dfs[b_no][i+k][s_no] == str(1) or dfs[b_no][i+k][s_no] == str(3)):
            k += 1
            forward_counter += 1
        if forward_counter > 1 and back_counter > 1:
            seg_array.append(back_counter + forward_counter + 1)

if len(seg_array) == 0:
    print('0 patterns found.')
else:
    print('Number of Hydrophobic segments with 1 OEGMA: ' + str(len(seg_array)))
    print('Length of Hydrophobic segments with 1 OEGMA: ' + str(seg_array))

In [ ]:
#sample = [1, 2, 3, 4, 1, 1, 2, 1, 3, 1, 3, 1, 3, 2, 3, 3, 4, 4, 2, 1, 2, 4, None, None, None]
sample = [1, 1, 1, 3, 4, 4, 4, 4, 1, 1, 2, 3, 1, 3, 2, 1, 1, 4, 2, 4, 1, 1, 2, 1, 3, None, None]

In [ ]:
#specific segment pattern search mon2mon level (still working on it)
oegma_counter = 0
seg_len = 0
seg_array = []
pos_array = list()

for i in range(len(sample)):
    if sample[i] == 2:
        back_counter = 0
        forward_counter = 0
        j = 1
        k = 1
        while (sample[i-j] == 1 or sample[i-j] == 3) and i-j >= 0:
            j += 1
            back_counter += 1
        while i+k <= len(sample)-1 and (sample[i+k] == 1 or sample[i+k] == 3):
            k += 1
            forward_counter += 1
        if forward_counter > 1 and back_counter > 1:
            seg_array.append(back_counter + forward_counter + 1)

print(seg_array)

In [ ]:
### SLIDING WINDOW ANALYSIS OF POLYMERS
# Set window length:
WIN_LENGTH = 10

# create a new list bins, to replace amino acids by corresponding hydrophobicity over the whole sequence 
def translate(chars, HLBs):
    bins = []
    for i in range(len(chars)):
        bins.append(HLBs[int(chars[i])-1])
    return bins

# iterate an iterable sequence by the number of size per time and create a list of sliding_Arrays for that
def window( iterable, size ):
    sliding_array=[]
    iters = tee(iterable, int(size))
    for i in range(1, int(size)):
        for each in iters[i:]:
            next(each, None)
    for each in zip(*iters):
        sliding_array.append(list(each))
    return sliding_array

# create a dictionary to keep the medium coordinate and average value for each sliding window
def seg_analysis(sliding_arrays,win_length):
    pos_val={}  
    int_pos = int(win_length) // 2 +1
    for each in sliding_arrays:
        ave_value=np.mean(each)
        pos_val[int_pos]=ave_value
        int_pos+=1
    return pos_val

#input a dictionary including positions and average value of a sliding arrays, output a plot
def win_plot(pos_val, pro_name, win_length, inverse=False):
    # sorted by key, return a list of tuples
    lists = list(sorted(pos_val.items())) 
    # unpack a list of pairs into two tuples
    x, y=zip(*lists)
    # Create a Figure
    fig =plt.figure(figsize=(8,8))
    # Set up Axes
    ax= fig.add_subplot(111)
    if inverse:
        plt.ylim((5,13))
        plt.gca().invert_yaxis()
    ax.scatter(x, y)
    ax.plot(x, y)
    #ax.set_xlim(0, 60)
    ax.set(title= "Hydropathy plot of " + str(pro_name) + " ,window length = " + str(win_length), xlabel="Center AA in AAs window", ylabel="Average Hydrophobicity/Window")
    plt.savefig("outputs/" + str(pro_name) + "_" + str(win_length) + ".png",transparent =True)
    plt.show()


# ACTUAL SCRIPT.
for m in tqdm_notebook(range(len(csv_files))):
    
    filename = csv_files[m].replace(path, '').replace('/', '').replace('.csv', '')

    mat = list()
    for seq in tqdm_notebook(seqs[m]):
        bins = translate(seq, HLBs)
        sliding_arrays = window(bins, WIN_LENGTH)
        pos_val = seg_analysis(sliding_arrays, WIN_LENGTH)
        lists = list(sorted(pos_val.items()))
        x,y = zip(*lists)

        mat.append(y)
            
    df = pd.DataFrame(mat)
    #r"outputs/HLB_%s.xlsx"%filename
    df.to_csv(r'outputs/%s_%s_%d.csv'% (filename,"HLB",int(WIN_LENGTH)),index=False)


In [ ]:
batch_num = 2
sequence_num = 1982

filename = csv_files[m].replace(path, '').replace('/', '').replace('.csv', '')
i = 0
for seq in tqdm_notebook(seqs[m]):
    if i == sequence_num:
        bins = translate(seq, HLBs)
        sliding_arrays = window(bins, WIN_LENGTH)
        pos_val = seg_analysis(sliding_arrays, WIN_LENGTH)
        win_plot(pos_val, filename, WIN_LENGTH)
    i = i + 1

## seq2seq level
Inter-chain variance within a batch
- Histogram of chemical heterogeneity: distribution around feed ratio
- box-whisker sliding window
- batch summed statistics (histogram)
- Specific segment pattern seq2seq distribution (i.e. 1 hydrophilic mon in hydrophobic segment)

In [ ]:
#Seq2Seq Inputs: ENTER HERE
window_len = 10 #specifies which window length HLB data to be analyzed (must be in outputs file)
batch_no = 9 #specifies batch number to be analyzed

### Plot histogram with KDE fit of composition on each individual monomer chain
Also get statistics for the plot above (peak height, stdev & KDE fit), calculate Full Width at Half Maximum (FWHM = 2*sqrt(2*ln(2))* stdev ~= 2.355*stdev ) & FWHM normalized by feeding fraction, then plot batch2batch

In [ ]:
### Histograms of composition on each individual monomer chain:

N_MONS = 4 # number of unique monomers
RUN_ONLY_SUBSET = False # set False if you want to run all sequences

FWHMs = []
FWHM_norms = []

for m in range(len(csv_files)):
    seq = []
    
    if RUN_ONLY_SUBSET:
        if m not in subset_inds:
            continue
    
    with open(csv_files[m], mode = 'r') as file:
        csvFile = csv.reader(file)
        i = 0
        for lines in csvFile:
            i = i + 1
            seq.append(lines)
            #print(lines)

    seqlen = np.zeros(i)
    seq_comp = np.zeros([i,4])

    for j in range(0,i):
        seqlen[j] = len(seq[j])
        for k in range(4):
            seq_comp[j][k] = seq[j].count(str(k+1))/seqlen[j]
            
    print(csv_files[m].replace(path, '').replace('/', '').replace('.csv', ''))
    FWHM = np.zeros(N_MONS)
    FWHM_norm = np.zeros(N_MONS)
    for i in range(N_MONS):
        #MMA_x, MMA_y = sns.distplot(seq_comp[:,0], label="MMA").get_lines()[0].get_data()
        FWHM[i] = 2.355*np.std(seq_comp[:,i])
        FWHM_norm[i] = 2.355*np.std(seq_comp[:,i])/np.mean(seq_comp[:,i])
    print(FWHM)
    print(FWHM_norm)
    FWHMs.append(FWHM)
    FWHM_norms.append(FWHM_norm)

    plt.figure(num=None, figsize=(12, 8), dpi=80, facecolor='w', edgecolor='k')
    sns.set_style(style='white') #style='white' or 'darkgrid'
    plt.xlabel("Feeding monomer fraction")
    plt.ylabel("Frequency")
    sns.distplot(seq_comp[:,0], label="MMA")
    sns.distplot(seq_comp[:,1], label="OEGMA500")
    sns.distplot(seq_comp[:,2], label="EHMA")
    sns.distplot(seq_comp[:,3], label="SPMA")
    plt.legend()
    plt.show()

    plt.cla()   # Clear axis
    plt.clf()   # Clear figure

    """ # (same plot but using plt instead of sns)
    plt.figure(num=None, figsize=(12, 8), dpi=80, facecolor='w', edgecolor='k')
    plt.xlabel("chain composition fraction", fontsize=15)
    #plt.ylabel("asdf")
    plt.hist(seq_comp[:,0], label="MMA")
    plt.hist(seq_comp[:,1], label="OEGMA500")
    plt.hist(seq_comp[:,2], label="EHMA")
    plt.hist(seq_comp[:,3], label="SPMA")
    plt.legend(fontsize=15)
    plt.show()
    """

FWHMs = np.array(FWHMs)
FWHM_norms = np.array(FWHM_norms)

In [ ]:
#get HLB files of a particular window length
path = "outputs"
hlb_csv_files = []
for file in os.listdir(path):
    if file.endswith("HLB_%d.csv"% (int(window_len))):
        hlb_csv_files.append(os.path.join(path, file))
hlb_csv_files = sorted(hlb_csv_files)
print("Number of sequence files:", len(hlb_csv_files))
print(hlb_csv_files)

In [ ]:
# read sliding window analysis csv files and convert to pd dataframes
dfs_hlb = []
seq_lens_hlb = []
batch_ind = batch_no - 1

for m in range(len(hlb_csv_files)):
    seq = []
    row_size = []
    
    with open(hlb_csv_files[m], mode = 'r') as file:
        csvFile = csv.reader(file)
        i = 0
        for lines in csvFile:
            i = i + 1
            if i > 1:
                seq.append(lines)
                row_size.append(len(lines))
        df = pd.DataFrame(seq)
        dfs_hlb.append(df)
        seq_lens_hlb.append(row_size)

rows_size = len(dfs_hlb[batch_ind])
cols_size = len(dfs_hlb[batch_ind].columns)

rows_size = len(dfs_hlb[batch_ind])
print("rows size:", rows_size)
cols_size = len(dfs_hlb[batch_ind].columns)
print("cols size:", cols_size)

In [ ]:
#Box Whisker Plot of Sliding Window Analaysis Data

pos_data = list()
for i in range(cols_size):
    datacol = [int(float((y_))) for y_ in dfs_hlb[batch_ind][i] if str(y_) != '']
    pos_data.append(datacol)
                      
fig = plt.figure(figsize =(15, 7))
ax = fig.add_axes([0, 0, 1, 1])
ax.boxplot(pos_data, whis=(0,100))
plt.xlabel("Sequence position")
plt.ylabel("Window average HLB")
plt.xticks([])
#plt.xticks(np.arange(0,cols_size, step=20)) #TODO: fix x axis marks
plt.show()

In [ ]:
# binarize data and save it to output csv.

binarized = []
for i in range(rows_size):
    binarized_seq = []
    for j in range(cols_size):
        if dfs_hlb[batch_ind][j][i] == '':
            binarized_seq.append(2)
        elif int(float(dfs_hlb[batch_ind][j][i])) > 9:
            binarized_seq.append(1)
        elif int(float(dfs_hlb[batch_ind][j][i])) <= 9:
            binarized_seq.append(0)
    binarized.append(binarized_seq)

binarized_df = pd.DataFrame(binarized)
binarized_df.to_csv(hlb_csv_files[batch_ind] + '_binarized.csv', index=False)

In [ ]:
#hydrophilic and hydrophobic segments count
hydrophob_li = list()
hydrophil_li = list()
for i in tqdm_notebook(range(rows_size)):
    hydrophobl = np.zeros(cols_size)
    hydrophilbl = np.zeros(cols_size)
    counterpho = 0
    counterphil = 0
    for j in range(cols_size):
        if binarized_df[j][i] == 0:
            counterpho += 1
            if counterphil > 0:
                hydrophilbl[counterphil-1] += 1
            counterphil = 0
            continue
        elif binarized_df[j][i] == 1:
            counterphil += 1
            if counterpho > 0:
                hydrophobl[counterpho-1] += 1
            counterpho = 0
            continue
        else:
            if counterpho > 0:
                hydrophobl[counterpho-1] += 1
            if counterphil > 0:
                hydrophilbl[counterphil-1] += 1
            break
    hydrophob_li.append(hydrophobl)
    hydrophil_li.append(hydrophilbl)

hydrophobic_df = pd.DataFrame(hydrophob_li)
hydrophilic_df = pd.DataFrame(hydrophil_li)

In [ ]:
hydrophobic_i = len(hydrophobic_df.columns)
hydrophilic_i = len(hydrophilic_df.columns)
graph1_phob = []
graph1_phil = []

# sum the hydrophobic counts
for i in range(hydrophobic_i):
    a = np.sum(hydrophobic_df[i])
    graph1_phob.append(a)

# sum the hydrophilic counts
for i in range(hydrophilic_i):
    b = np.sum(hydrophilic_df[i])
    graph1_phil.append(b)
    
# go backwards from both until you find the first non-zero entry and truncate there
for i in range(1,len(graph1_phob)):
    if (graph1_phob[-i] != 0.0):
        graph1_phob = graph1_phob[:-i]
        break

for i in range(1,len(graph1_phil)):
    if (graph1_phil[-i] != 0.0):
        graph1_phil = graph1_phil[:-i]
        break

# create x-axis for both
x_phob_1 = np.zeros(len(graph1_phob))
x_phil_1 = np.zeros(len(graph1_phil))

for i in range(len(graph1_phob)):
    x_phob_1[i] = i+1

for i in range(len(graph1_phil)):
    x_phil_1[i] = i+1

In [ ]:
### PLOT hydrophobic segments
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(x_phob_1,graph1_phob, width = 1)
plt.xlabel("segment length")
plt.ylabel("number of segments")
plt.title("Hydrophobic Segments")
plt.show()

In [ ]:
### PLOT hydrophilic segments
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(x_phil_1,graph1_phil, width = 1)
plt.xlabel("segment length")
plt.ylabel("number of segments")
plt.title("Hydrophilic Segments")
plt.show()

In [ ]:
### FULL HYDROPHOBIC DATASET (non-zero)
phobic_length_data = []
for i in range(len(graph1_phob)):
    phobic_length_data.append([ y_ for y_ in list(hydrophobic_df[i]) if y_ != 0.0])
        
fig = plt.figure(figsize =(12, 7))
#ax = fig.add_axes([0, 1, 2, 1])
ax = fig.add_axes([0, 0, 1, 1])
ax.boxplot(phobic_length_data, whis=(0,100))
plt.xlabel("Hydrophobic Segment Length")
plt.ylabel("Non-zero counts")
#plt.xticks([])
#plt.xticks(np.arange(0,cols_size, step=20)) #TODO: fix x axis marks
plt.show()

#  just hydrophobic segment lengths up to 10
phobic_length_data2 = []
for i in range(12):
    phobic_length_data2.append([ y_ for y_ in list(hydrophobic_df[i]) if y_ != 0.0])

fig = plt.figure(figsize =(12, 7))
#ax2 = fig.add_axes([0, 0, 2, 1])
ax = fig.add_axes([0, 0, 1, 1])
ax.boxplot(phobic_length_data2, whis=(0,100))
plt.xlabel("Hydrophobic Segment Length")
plt.ylabel("Non-zero counts")
#plt.xticks([])
#plt.xticks(np.arange(0,cols_size, step=20)) #TODO: fix x axis marks
plt.show()

In [ ]:
### FULL HYDROphilic DATASET (non-zero)
philic_length_data = []
for i in range(len(graph1_phil)):
    philic_length_data.append([ y_ for y_ in list(hydrophilic_df[i]) if y_ != 0.0])
        
fig = plt.figure(figsize =(12, 7))
#ax = fig.add_axes([0, 1, 2, 1])
ax = fig.add_axes([0, 0, 1, 1])
ax.boxplot(philic_length_data, whis=(0,100))
plt.xlabel("Hydrophilic Segment Length")
plt.ylabel("Non-zero counts")
#plt.xticks([])
#plt.xticks(np.arange(0,cols_size, step=20)) #TODO: fix x axis marks

#  just hydrophilic segment lengths up to 10
philic_length_data2 = []
for i in range(12):
    philic_length_data2.append([ y_ for y_ in list(hydrophilic_df[i]) if y_ != 0.0])

fig = plt.figure(figsize =(12, 7))
#ax2 = fig.add_axes([0, 0, 2, 1])
ax = fig.add_axes([0, 0, 1, 1])
ax.boxplot(philic_length_data2, whis=(0,100))
plt.xlabel("Hydrophilic Segment Length")
plt.ylabel("Non-zero counts")
#plt.xticks([])
#plt.xticks(np.arange(0,cols_size, step=20)) #TODO: fix x axis marks
plt.show()

In [ ]:
### Specific segment pattern search (seq2seq level)
b_no = batch_no

oegma_counter = 0
seg_len_list = []
seg_array_b = list()
seg_arr = []
allseg_array = []

for s_no in range(len(dfs[b_no])):
    seg_arr = []
    for i in range(len(dfs[b_no].iloc[s_no])):
        if dfs[b_no][i][s_no] == str(2):
            back_counter = 0
            forward_counter = 0
            j = 1
            k = 1
            while i-j >= 0 and (dfs[b_no][i-j][s_no] == str(1) or dfs[b_no][i-j][s_no] == str(3)):
                j += 1
                back_counter += 1
            while i+k <= len(dfs[b_no].iloc[s_no])-1 and (dfs[b_no][i+k][s_no] == str(1) or dfs[b_no][i+k][s_no] == str(3)):
                k += 1
                forward_counter += 1
            if forward_counter > 1 and back_counter > 1:
                seg_arr.append(back_counter + forward_counter + 1)
                allseg_array.append(back_counter + forward_counter + 1)
    seg_len_list.append(len(seg_arr))
    seg_array_b.append(seg_arr)

#seg_array_b --> list of lengths of Hydrophobic segments with 1 OEGMA per sequence in specified batch
#seg_len_list --> array of number of Hydrophobic segments with 1 OEGMA per sequence in specified batch
 

In [ ]:
#analyzes specific segment search data
Avg_NumSeg = np.mean(np.array(seg_len_list))
StDev_NumSeg = np.std(np.array(seg_len_list))
Avg_LenSeg = []
StDev_LenSeg = []

for i in range(len(seg_array_b)):
    avg = np.mean(np.array(seg_array_b[i]))
    sdev = np.std(np.array(seg_array_b[i]))
    Avg_LenSeg.append(avg)
    StDev_LenSeg.append(sdev)

print("Average Number of Hydrophobic segments with 1 Oegma per sequence in batch: " + str(Avg_NumSeg))
print("StDev of Number of Hydrophobic segments with 1 Oegma per sequence in batch: " + str(StDev_NumSeg))
#Uncomment following lines for length data per sequence
#print("Average lengths of Hydrophobic segments with 1 OEGMA per sequence in batch: " + str(Avg_LenSeg))
#print("StDev of lengths of Hydrophobic segments with 1 Oegma per sequence in batch: " + str(StDev_LenSeg))

fig = plt.figure()
plt.xlabel("segment length")
plt.ylabel("# of segments")
plt.title("Hydrophobic Segments with 1 OEGMA")
plt.hist(allseg_array, bins=32)

## batch2batch level
Average batch statistics variance
- repeats (how random is the MC)
- Compare full width at half maximum  normalized  by monomer feeding  fraction (nFWHM)  across batches
- Compare hydropathy,  segment length  & specific  segment  pattern plots across batches

### Plot batch2batch variation (FWHM)
Add 2D array obtained in previous cell to sub-set dataframe (sub_df), the plot a scatter with line of best fit!

In [ ]:
for i in range(4):
    df_prop['FWHM_' + str(i)] = FWHM_norms[:,i]
    
fig, ax = plt.subplots(1, 1, figsize=(10,10))
sns.set_style(style='white') #style='white' or 'darkgrid'
ax = sns.regplot(x="Avg DP", y="FWHM_0", data=df_prop, label='MMA')
ax = sns.regplot(x="Avg DP", y="FWHM_1", data=df_prop, label='OEGMA')
ax = sns.regplot(x="Avg DP", y="FWHM_2", data=df_prop, label='EHMA') 
ax = sns.regplot(x="Avg DP", y="FWHM_3", data=df_prop, label='SPMA')
ax.set_ylabel('FWHM')
#ax.set(ylim=(0, 1))
plt.legend(loc='best')
plt.show()